In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

VER = 203
USE = 'carts'

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
users = pd.read_parquet(f'../../data/train_with_features/train_v174_X.pqt')
print( users.shape )
users.head()

(23388700, 226)


,user,item,pos,wgt,carts,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7,buy_ratio8,count_item8,count_user8,hour_mean8,day_mean8,repeat8,buy_ratio_std8,hour_std8,day_std8,prev8,next8,orders8,carts8,order_repeat8,cart_repeat8,buy_ratio9,count_item9,count_user9,hour_mean9,day_mean9,repeat9,buy_ratio_std9,hour_std9,day_std9,prev9,next9,orders9,carts9,order_repeat9,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10,z10,z11,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11,buy_ratio12,count_item12,count_user12,hour_mean12,day_mean12,repeat12,buy_ratio_std12,hour_std12,day_std12,buy_ratio13,count_item13,count_user13,hour_mean13,day_mean13,repeat13,buy_ratio_std13,hour_std13,day_std13,buy_ratio14,count_item14,unique_item14,hour_mean14,day_mean14,repeat14,buy_ratio_std14,hour_std14,day_std14,buy_ratio15,count_item15,unique_item15,hour_mean15,day_mean15,repeat15,buy_ratio_std15,hour_std15,day_std15,last_0,last_1,last_2,last_3,last_4,first_0,first_1,first_2,first_3,first_4,buy_0,buy_1,buy_2,buy_3,buy_4,e_mean_0,e_std_0,e_mean_1,e_std_1,e_mean_2,e_std_2,e_mean_3,e_std_3,click_oof
0,11098533,1165015,0,-1.0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.600000,-1.0,2.200000,-1.0,-1.0,1.500000,1.600000,1.500000,1.500000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.116279,86.0,58.0,1.482759,0.357067,6.755245,6.755245,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.307692,1.0,-1.0,1.0,-1.0,1.0,-1.0,13.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,7.0,4.750940,4.750940,0.136364,154.0,99.0,56034.429688,321966.906250,1.555556,0.397183,17312.884766,153317.859375,0.250000,16.0,10.0,57562.187500,300562.18750,1.600000,0.577350,22206.015625,82783.578125,10.0,9.0,1.0,2.0,1.000000,1.000000,0.117647,17.0,13.0,38353.824219,373789.125,1.307692,0.332106,15429.47168,13270.989258,40415,1,40414,1.000000,0.987719,0.947001,-1.0,-1.0,-1.0,0.076190,105.0,78.0,56847.656250,318516.218750,1.346154,0.300488,16512.648438,167820.515625,77.0,69.0,1.0,6.0,3.0,6.000000,0.159574,94.0,64.0,67433.125000,314684.18750,1.468750,0.422567,11249.509766,159400.546875,71.0,65.0,2.0,11.0,2.5,2.142857,0.100000,60.0,38.0,38176.464844,333376.468750,1.578947,0.354152,6817.803223,143838.656250,38.0,43.0,1.0,4.0,-1.0,-1.000000,1.0,2.0,2.0,79339.5,338539.5,1.0,0.0,195.868576,195.868576,0.987719,1.0,1.166667,18.0,13.0,62896.667969,336496.656250,1.384615,0.383482,16853.607422,144629.515625,1.333333,3.0,2.0,67674.335938,269274.343750,1.500000,0.577350,24975.738281,24909.345703,1.333333,3.0,2.0,67674.335938,269274.34375,1.5,0.57735,24975.738281,24909.345703,0.117647,17.0,13.0,38353.824219,373789.125,1.307692,0.332106,15429.47168,13270.989258,1.0,2.0,2.0,79339.5,338539.5,1.0,0.0,195.868576,195.868576,1.000000,0.732554,0.788323,0.805929,0.637763,0.739908,0.805929,0.805929,0.656059,0

# CatBoost

In [4]:
import catboost as cat
from sklearn.model_selection import GroupKFold
from catboost import CatBoostRanker, CatBoostClassifier, Pool
print('CatBoost version',cat.__version__)

FOLDS=5
SEED = 42

CatBoost version 1.0.6


In [5]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

223
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [6]:
%%time
if 0:
    for c in users.columns[2:]:
        print(c,', ',end='')
        if str(users[c].dtype)=='float32':
            users[c] = users[c].astype('float16')
        if str(users[c].dtype)=='int32':
            users[c] = users[c].astype('int16')

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 12.9 µs


In [7]:
# RMSE QueryRMSE PairLogit, PairLogitPairwise, YetiRank, YetiRankPairwise

In [8]:
%%time
oof = np.zeros(len(users))

skf = GroupKFold(n_splits=FOLDS)
for fold,(train_idx, valid_idx) in enumerate(skf.split(
            X=users, y=users[USE], groups=users.user )):
    
    print('#'*25)
    print('### Fold',fold+1)
    print('### Train size',len(train_idx),'Valid size',len(valid_idx))
    print('#'*25)
    
    model = CatBoostRanker(iterations=5_000,
                              random_state=SEED,
                              max_depth=4,
                              #max_leaves=32,
                              learning_rate=0.1,
                              #subsample=0.7,
                              task_type='GPU',
                              #custom_metric = 'MAP:top=50',
                              #eval_metric = 'MAP',
                              #colsample_bylevel=0.5,
                              #thread_count=16,
                              #scale_pos_weight=10
                              loss_function='PairLogitPairwise'
                           )
    
    train = Pool(
        data = users.loc[train_idx, FEATURES],
        label = users.loc[train_idx, USE],
        group_id = users.loc[train_idx, 'user']
    )
    
    valid = Pool(
        data = users.loc[valid_idx, FEATURES],
        label = users.loc[valid_idx, USE],
        group_id = users.loc[valid_idx, 'user']
    )
    
    model.fit(train,
             verbose=100,
             early_stopping_rounds=250,
             eval_set=valid,
             )
    model.save_model(f'../../data/models/CAT_fold{fold}_{USE}_v{VER}.cat')
    

    if 0:
        feature_importance = model.feature_importances_
        sorted_idx = np.argsort(feature_importance)
        fig = plt.figure(figsize=(10, 20))
        plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
        plt.yticks(range(len(sorted_idx)), np.array(FEATURES)[sorted_idx])
        plt.title('Feature Importance')
        plt.show()
          
    # INFER OOF FOLD K
    oof_preds = model.predict(users.loc[valid_idx, FEATURES])
    oof[valid_idx] = oof_preds
    
    del model
    gc.collect()

#########################
### Fold 1
### Train size 18710900 Valid size 4677800
#########################
0:	learn: 0.6314414	test: 0.6313282	best: 0.6313282 (0)	total: 39.8ms	remaining: 3m 18s
100:	learn: 0.2192755	test: 0.2193710	best: 0.2193710 (100)	total: 3.54s	remaining: 2m 51s
200:	learn: 0.2143648	test: 0.2151508	best: 0.2151508 (200)	total: 6.93s	remaining: 2m 45s
300:	learn: 0.2120186	test: 0.2134061	best: 0.2134061 (300)	total: 10.3s	remaining: 2m 40s
400:	learn: 0.2103628	test: 0.2124353	best: 0.2124353 (400)	total: 13.7s	remaining: 2m 37s
500:	learn: 0.2090182	test: 0.2117253	best: 0.2117253 (500)	total: 17.3s	remaining: 2m 35s
600:	learn: 0.2078758	test: 0.2112289	best: 0.2112250 (599)	total: 20.8s	remaining: 2m 32s
700:	learn: 0.2068452	test: 0.2108689	best: 0.2108689 (700)	total: 24.7s	remaining: 2m 31s
800:	learn: 0.2059170	test: 0.2105640	best: 0.2105640 (800)	total: 28.5s	remaining: 2m 29s
900:	learn: 0.2050398	test: 0.2102911	best: 0.2102892 (896)	total: 32.2s	remai

2800:	learn: 0.1919670	test: 0.2112790	best: 0.2112563 (2733)	total: 2m 4s	remaining: 1m 37s
2900:	learn: 0.1914305	test: 0.2112615	best: 0.2112470 (2851)	total: 2m 9s	remaining: 1m 33s
3000:	learn: 0.1908997	test: 0.2112769	best: 0.2112470 (2851)	total: 2m 14s	remaining: 1m 29s
3100:	learn: 0.1903690	test: 0.2112928	best: 0.2112470 (2851)	total: 2m 19s	remaining: 1m 25s
bestTest = 0.2112469732
bestIteration = 2851
Shrink model to first 2852 iterations.
#########################
### Fold 4
### Train size 18711000 Valid size 4677700
#########################
0:	learn: 0.6314715	test: 0.6312317	best: 0.6312317 (0)	total: 39.5ms	remaining: 3m 17s
100:	learn: 0.2194213	test: 0.2186632	best: 0.2186632 (100)	total: 3.57s	remaining: 2m 53s
200:	learn: 0.2145146	test: 0.2144331	best: 0.2144331 (200)	total: 7.45s	remaining: 2m 57s
300:	learn: 0.2121752	test: 0.2127828	best: 0.2127828 (300)	total: 11.6s	remaining: 3m
400:	learn: 0.2104978	test: 0.2118263	best: 0.2118263 (400)	total: 15.8s	remain

# Make Sub

In [9]:
sub = users[['user','item']].copy()
sub['p'] = oof
#sub = cudf.DataFrame(sub)
sub = sub.sort_values(['user','p'],ascending=[True,False])
print( sub.shape )
sub.head()

(23388700, 3)


,user,item,p
0,11098533,1165015,4.659765
3,11098533,1074173,3.856705
10,11098533,1309900,3.634723
5,11098533,833149,3.503740
9,11098533,935297,3.332137


In [10]:
sub = sub.reset_index(drop=True)

In [11]:
sub['n'] = sub.groupby('user').item.cumcount().astype('int8')

In [12]:
sub.head()

,user,item,p,n
0,11098533,1165015,4.659765,0
1,11098533,1074173,3.856705,1
2,11098533,1309900,3.634723,2
3,11098533,833149,3.503740,3
4,11098533,935297,3.332137,4


In [13]:
sub = sub.loc[sub.n<20]
print( sub.shape )

(4677740, 4)


In [14]:
#sub = sub.to_pandas()

In [15]:
%%time
sub2 = sub.groupby('user').item.apply(list)

CPU times: user 3.97 s, sys: 84.5 ms, total: 4.06 s
Wall time: 4.05 s


In [16]:
sub2 = sub2.to_frame().reset_index()

In [17]:
%%time
sub2.item = sub2.item.apply(lambda x: " ".join(map(str,x)))

CPU times: user 891 ms, sys: 31 ms, total: 922 ms
Wall time: 920 ms


In [18]:
sub2.columns = ['session_type','labels']

In [19]:
sub2.session_type = sub2.session_type.astype('str')+ f'_{USE}'

In [20]:
sub2.head()

,session_type,labels
0,11098533_carts,1165015 1074173 1309900 833149 935297 978918 1...
1,11098534_carts,223062 908024 1607945 1449202 1342293 1300062 ...
2,11098538_carts,1263747 1550143 1317439 1172033 987426 1684753...
3,11098542_carts,1571098 1771977 708196 1025387 594491 875906 2...
4,11098543_carts,1168557 1458173 114619 1411709 1295782 1039959...


In [21]:
submission = sub2

In [22]:
def fill_up(x):
    return []

In [23]:
%%time
submission['session'] = submission.session_type.apply(lambda x: int(x.split('_')[0]))
submission['type'] = submission.session_type.apply(lambda x: x.split('_')[1])
submission.labels = submission.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

test_labels = pd.read_parquet('/raid/Kaggle/otto/valid/test_labels.parquet')
test_labels = test_labels.loc[test_labels['type']==USE]

test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
test_labels.loc[test_labels.labels.isna(),'labels'] =\
    test_labels.loc[test_labels.labels.isna(),'labels'].map(fill_up)

test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)

recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 

score = (recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()
print( score , recall_per_type)

# carts    0.441457

0.1326577412651219 type
carts    0.442192
dtype: float64
CPU times: user 9.97 s, sys: 245 ms, total: 10.2 s
Wall time: 9.99 s
